# Mandatory Challenge
## Context
You work in the data analysis team of a very important company. On Monday, the company shares some good news with you: you just got hired by a major retail company! So, let's get prepared for a huge amount of work!

Then you get to work with your team and define the following tasks to perform:   
1. You need to start your analysis using data from the past.  
2. You need to define a process that takes your daily data as an input and integrates it.  

You are in charge of the second part, so you are provided with a sample file that you will have to read daily. To complete you task, you need the following aggregates:
* One aggregate per store that adds up the rest of the values.
* One aggregate per item that adds up the rest of the values.

You can import the dataset `warehouse_and_retail_sales` from Ironhack's database. 

## Your task
Therefore, your process will consist of the following steps:
1. Read the sample file that a daily process will save in your folder. 
2. Clean up the data.
3. Create the aggregates.
4. Write three tables in your local database: 
    - A table for the cleaned data.
    - A table for the aggregate per supplier.
    - A table for the aggregate per item.

## Instructions
* Read the csv you can find in Ironhack's database.
* Clean the data and create the aggregates as you consider.
* Create the tables in your local database.
* Populate them with your process.

In [1]:
# Import libraries
import pandas as pd
import os
import numpy as np
from scipy import stats

In [2]:
# Import the dataset
data = pd.read_csv('./Warehouse_and_Retail_Sales.csv')

In [3]:
# View the dataset
data.head(61)

,YEAR,MONTH,SUPPLIER,ITEM CODE,ITEM DESCRIPTION,ITEM TYPE,RETAIL SALES,RETAIL TRANSFERS,WAREHOUSE SALES
0,2017,4,ROYAL WINE CORP,100200,GAMLA CAB - 750ML,WINE,0.0,1.0,0.0
1,2017,4,SANTA MARGHERITA USA INC,100749,SANTA MARGHERITA P/GRIG ALTO - 375ML,WINE,0.0,1.0,0.0
2,2017,4,JIM BEAM BRANDS CO,10103,KNOB CREEK BOURBON 9YR - 100P - 375ML,LIQUOR,0.0,8.0,0.0
3,2017,4,HEAVEN HILL DISTILLERIES INC,10120,J W DANT BOURBON 100P - 1.75L,LIQUOR,0.0,2.0,0.0
4,2017,4,ROYAL WINE CORP,101664,RAMON CORDOVA RIOJA - 750ML,WINE,0.0,4.0,0.0
...,...,...,...,...,...,...,...,...,...
56,2017,4,JIM BEAM BRANDS CO,10667,EFFEN VODKA - BLACK CHERRY - 375ML,LIQUOR,0.0,2.0,0.0
57,2017,4,JIM BEAM BRANDS CO,10668,MAKERS MARK BOURBON - 375ML,LIQUOR,0.0,27.0,0.0
58,2017,4,PRESTIGE BEVERAGE GROUP OF MD LLC,10673,CINNABON CINNAMON CREAM LIQ - 750ML,LIQUOR,0.0,1.0,0.0
59,2017,4,MOET HENNESSY USA,106739,VEUVE CLICQUOT DEMI SEC - 375ML,WINE,0.0,1.0,0.0


In [4]:
# Identifying datatypes
data.dtypes

YEAR                  int64
MONTH                 int64
SUPPLIER             object
ITEM CODE            object
ITEM DESCRIPTION     object
ITEM TYPE            object
RETAIL SALES        float64
RETAIL TRANSFERS    float64
WAREHOUSE SALES     float64
dtype: object

In [5]:
# View the data shape
data.shape

(128355, 9)

In [6]:
# Identify columns with empty values
data_empty = data.isnull().sum()
print(data_empty[data_empty>0])

SUPPLIER     24
ITEM TYPE     1
dtype: int64


In [7]:
# There are only a few empty values for Supplier so we drop them
drop_rows = data.loc[data['SUPPLIER'].isnull()].index
data = data.drop(drop_rows, axis = 0)

In [8]:
# There are only one empty value for Item Type so we drop it
drop_rows = data.loc[data['ITEM TYPE'].isnull()].index
data = data.drop(drop_rows, axis = 0)

In [9]:
# View the data shape
data.shape

(128330, 9)

In [10]:
# Double check columns with empty values
data_empty = data.isnull().sum()
print(data_empty[data_empty>0])

Series([], dtype: int64)


In [11]:
# Identify duplicate records and drop them
data_before = len(data)
data.drop_duplicates()
data_after = len(data)
print('Number of duplicate records dropped: ', str(data_before - data_after))

Number of duplicate records dropped:  0


In [12]:
# Identify low variance
low_variance = []
for col in data._get_numeric_data():
    minimum = min(data[col])
    ninety_perc = np.percentile(data[col], 90)
    if ninety_perc == minimum:
        low_variance.append(col)
print(low_variance)

[]


In [13]:
if low_variance == []:
    print("There were no numerical values with low variance, so we did not drop any values")
else: 
    print("There were numerical values with low variance, but we decided not to drop any values")

There were no numerical values with low variance, so we did not drop any values


In [14]:
# Investigate descriptive statistics
data.describe().transpose()
stats = data.describe().transpose()
stats['IQR'] = stats['75%'] - stats['25%']
stats

,count,mean,std,min,25%,50%,75%,max,IQR
YEAR,128330.0,2017.206024,0.404449,2017.00,2017.0,2017.00,2017.00,2018.00,0.00
MONTH,128330.0,7.079225,3.645799,1.00,5.0,8.00,10.00,12.00,5.00
RETAIL SALES,128330.0,6.564316,28.927617,-6.49,0.0,0.33,3.25,1616.60,3.25
RETAIL TRANSFERS,128330.0,7.189561,30.642976,-27.66,0.0,0.00,4.00,1587.99,4.00
WAREHOUSE SALES,128330.0,22.681679,239.574553,-4996.00,0.0,1.00,4.00,16271.75,4.00


In [15]:
# Check for outliers
outliers = pd.DataFrame(columns = data.columns)
for col in stats.index:
    iqr = stats.at[col,'IQR']
    cutoff = iqr * 1.5
    lower = stats.at[col, '25%'] - cutoff
    upper = stats.at[col, '75%'] + cutoff
    results = data[(data[col] < lower) |
                  (data[col] > upper)].copy()
results['outlier'] = col
outlier = outliers.append(results)
outlier

print("It seems that Warehouse Sales has 18.213 outliers")

It seems that Warehouse Sales has 18.213 outliers


In [16]:
# Harmonize values - View all columns
print(set(data))

{'RETAIL TRANSFERS', 'MONTH', 'YEAR', 'WAREHOUSE SALES', 'RETAIL SALES', 'ITEM DESCRIPTION', 'ITEM CODE', 'ITEM TYPE', 'SUPPLIER'}


In [17]:
# Harmonize values - Item type
set(data['ITEM TYPE'])

{'BEER',
 'DUNNAGE',
 'KEGS',
 'LIQUOR',
 'NON-ALCOHOL',
 'REF',
 'STR_SUPPLIES',
 'WINE'}

In [18]:
# Harmonize values - Supplier
set(data['SUPPLIER'])

{'8 VINI INC',
 'A HARDY USA LTD',
 'A I G WINE & SPIRITS',
 'A VINTNERS SELECTIONS',
 'A&E INC',
 'A&W BORDERS LLC',
 'ADAMBA IMPORTS INTL',
 'AIKO IMPORTERS INC',
 'ALLAGASH BREWING COMPANY',
 'ALLIED IMPORTERS USA LTD',
 'ALTITUDE SPIRITS INC',
 'AMERICAN BEVERAGE CORPORATION',
 'AMERICAN BEVERAGE MARKETERS',
 'AMERICAN FIDELITY TRADING',
 'AMERICAN VINTAGE BEVERAGE INC',
 'ANHEUSER BUSCH INC',
 'ARCHER ROOSE LLC',
 'AREL GROUP WINE & SPIRITS',
 'ARIS A ZISSIS',
 'ARTISANS & VINES LLC',
 'ASAHI BEER USA INC',
 'ATLANTIC WINE & SPIRITS',
 'ATLAS BREW WORKS LLC',
 'AW DIRECT LLC',
 'AZIZ SHAFI TANNIC TONGUE',
 'BACARDI USA INC',
 'BACCHUS IMPORTERS LTD',
 'BACKUP BEVERAGE',
 'BANFI PRODUCTS CORP',
 'BANVILLE & JONES WINE MERCHANTS',
 'BARON FRANCOIS LTD',
 'BARREL ONE INC',
 'BASIGNANI WINERY',
 'BINDING BRAUEREI USA INC',
 'BLACK ANKLE VINEYARDS LLC',
 'BOND DISTRIBUTING CO',
 'BOORDY VINEYARDS',
 'BORVIN BEVERAGE',
 'BOSTON BEER CORPORATION',
 'BOUTIQUE VINEYARDS LLC',
 'BRONCO WINE

In [19]:
# Replace values - Supplier
data['SUPPLIER'] = data['SUPPLIER'].str.replace(',',' ')
# Check
set(data['SUPPLIER'])

{'8 VINI INC',
 'A HARDY USA LTD',
 'A I G WINE & SPIRITS',
 'A VINTNERS SELECTIONS',
 'A&E INC',
 'A&W BORDERS LLC',
 'ADAMBA IMPORTS INTL',
 'AIKO IMPORTERS INC',
 'ALLAGASH BREWING COMPANY',
 'ALLIED IMPORTERS USA LTD',
 'ALTITUDE SPIRITS INC',
 'AMERICAN BEVERAGE CORPORATION',
 'AMERICAN BEVERAGE MARKETERS',
 'AMERICAN FIDELITY TRADING',
 'AMERICAN VINTAGE BEVERAGE INC',
 'ANHEUSER BUSCH INC',
 'ARCHER ROOSE LLC',
 'AREL GROUP WINE & SPIRITS',
 'ARIS A ZISSIS',
 'ARTISANS & VINES LLC',
 'ASAHI BEER USA INC',
 'ATLANTIC WINE & SPIRITS',
 'ATLAS BREW WORKS LLC',
 'AW DIRECT LLC',
 'AZIZ SHAFI TANNIC TONGUE',
 'BACARDI USA INC',
 'BACCHUS IMPORTERS LTD',
 'BACKUP BEVERAGE',
 'BANFI PRODUCTS CORP',
 'BANVILLE & JONES WINE MERCHANTS',
 'BARON FRANCOIS LTD',
 'BARREL ONE INC',
 'BASIGNANI WINERY',
 'BINDING BRAUEREI USA INC',
 'BLACK ANKLE VINEYARDS LLC',
 'BOND DISTRIBUTING CO',
 'BOORDY VINEYARDS',
 'BORVIN BEVERAGE',
 'BOSTON BEER CORPORATION',
 'BOUTIQUE VINEYARDS LLC',
 'BRONCO WINE

In [20]:
# Export cleaned document to folder
data.to_csv('./Warehouse_and_Retail_Sales_Clean.csv')

In [21]:
data.describe()

,YEAR,MONTH,RETAIL SALES,RETAIL TRANSFERS,WAREHOUSE SALES
count,128330.000000,128330.000000,128330.000000,128330.000000,128330.000000
mean,2017.206024,7.079225,6.564316,7.189561,22.681679
std,0.404449,3.645799,28.927617,30.642976,239.574553
min,2017.000000,1.000000,-6.490000,-27.660000,-4996.000000
25%,2017.000000,5.000000,0.000000,0.000000,0.000000
50%,2017.000000,8.000000,0.330000,0.000000,1.000000
75%,2017.000000,10.000000,3.250000,4.000000,4.000000
max,2018.000000,12.000000,1616.600000,1587.990000,16271.750000


In [22]:
# Create dataframe for suppliers
data_supplier = data.copy()

In [23]:
# Aggregation per supplier that adds up the rest of the values.
data_supplier = data_supplier.groupby(['SUPPLIER'])['WAREHOUSE SALES', 'RETAIL SALES'].agg(['sum'])
data_supplier

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  


,WAREHOUSE SALES,RETAIL SALES
,sum,sum
SUPPLIER,,
8 VINI INC,1.00,2.78
A HARDY USA LTD,0.00,0.40
A I G WINE & SPIRITS,134.00,12.52
A VINTNERS SELECTIONS,29776.67,8640.57
A&E INC,0.00,11.52
...,...,...
WINEBOW INC,0.00,1.24
YOUNG WON TRADING INC,2528.90,1058.65


In [24]:
# Export cleaned document to folder
data_supplier.to_csv('./Warehouse_and_Retail_Sales_Supplier.csv')

In [25]:
# Create dataframe for item types
data_item = data.copy()

In [26]:
# Aggregation per item type that adds up the rest of the values.
data_item = data_item.groupby(['ITEM TYPE'])['WAREHOUSE SALES', 'RETAIL SALES'].agg(['sum'])
data_item

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  


,WAREHOUSE SALES,RETAIL SALES
,sum,sum
ITEM TYPE,,
BEER,2437617.32,209763.11
DUNNAGE,-45275.00,0.00
KEGS,43558.00,0.00
LIQUOR,33173.32,309847.85
NON-ALCOHOL,8656.72,8109.97
REF,0.00,281.34
STR_SUPPLIES,0.00,995.98
WINE,433009.47,313400.42


In [27]:
# Export cleaned document to folder
data_item.to_csv('./Warehouse_and_Retail_Sales_Item.csv')